In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [2]:
def product_lookup(itemNumbers):
    
    if isinstance(itemNumbers, int):
        itemNumbers = [itemNumbers]
        
    db = product_info[product_info.ItemNumber.isin(itemNumbers)]
        
    
    if db.shape[0] == 0:
        return "No item found"
    
    return db

def revenue_stitcher(invoices, itemNumbers, all_time = False):
    
    items = product_lookup(itemNumbers)

    if all_time:
        sales = invoices[invoices.ItemNumber.isin(itemNumbers)]

        new_df = pd.merge(items, sales)

        del items
        del sales

        new_df['ProfitPerBottle'] = new_df.StateBottleRetail - new_df.StateBottleCost
        new_df['ProfitPerCase'] = new_df.StateBottleRetail * new_df.Pack - new_df.StateCaseCost

        new_df['Overall_Revenue'] = new_df.BottlesSold * new_df.StateBottleRetail
        new_df['Overall_Profit'] = new_df.ProfitPerBottle * new_df.BottlesSold

        return new_df
    
    else:
        sales = invoices[invoices.ItemNumber.isin(itemNumbers)]

        sales = sales.pivot(index='ItemNumber', columns = 'Year', values = ['BottlesSold', 'UniqueTransactions']).fillna(0)

        sales.columns = [x[0]+"_"+x[1] for x in sales_pivot.columns.to_flat_index()]

        new_df = pd.merge(items, sales, left_on = 'ItemNumber', right_index = True)
        
        #tbd
        

In [3]:
alltime_sales = pd.read_csv('./data/alltime_sales.csv')
yearly_sales = pd.read_csv('./data/yearly_sales.csv')

In [2]:
product_info = pd.read_csv('./data/Iowa_Liquor_Products.csv')

In [3]:
product_info.columns

Index(['Item Number', 'Category Name', 'Item Description', 'Vendor',
       'Vendor Name', 'Bottle Volume (ml)', 'Pack', 'Inner Pack', 'Age',
       'Proof', 'List Date', 'UPC', 'SCC', 'State Bottle Cost',
       'State Case Cost', 'State Bottle Retail', 'Report Date'],
      dtype='object')

In [13]:
sum(product_info['Category Name'].str.contains('whiskey'))

0

In [37]:
idx = product_info.loc[product_info['Item Description'].str.contains('bucket', case = False) & product_info['Pack'] == 1, 'Pack'].index

In [38]:
idx

Int64Index([8202, 9554, 9793], dtype='int64')

In [42]:
product_info.loc[idx[0], 'Pack'] = 6
product_info.loc[idx[1], 'Pack'] = 6
product_info.loc[idx[2], 'Pack'] = 6

In [43]:
price = 51.6/6
retail = 77.4/6

In [47]:
product_info.loc[idx[0], 'State Bottle Cost'] = price
product_info.loc[idx[1], 'State Bottle Cost'] = price
product_info.loc[idx[2], 'State Bottle Cost'] = price

In [48]:
product_info.loc[idx[0], 'State Bottle Retail'] = retail
product_info.loc[idx[1], 'State Bottle Retail'] = retail
product_info.loc[idx[2], 'State Bottle Retail'] = retail

In [49]:
product_info.loc[idx]

,Item Number,Category Name,Item Description,Vendor,Vendor Name,Bottle Volume (ml),Pack,Inner Pack,Age,Proof,List Date,UPC,SCC,State Bottle Cost,State Case Cost,State Bottle Retail,Report Date
8202,100413,Whiskey Liqueur,Fireball Cinnamon Whiskey Party Bucket,421,SAZERAC COMPANY INC,50,6,120,0,66,04/01/2020,8.800403e+10,17503006845144,8.6,51.6,12.9,04/01/2022
9554,101539,Temporary & Specialty Packages,Southern Comfort Party Bucket Mini,421,SAZERAC COMPANY INC,50,6,120,0,100,03/01/2022,8.800404e+10,17503006845144,8.6,51.6,12.9,04/01/2022
9793,100861,American Cordials & Liqueur,99 Party Bucket Minis,421,SAZERAC COMPANY INC,50,6,120,0,99,09/01/2021,8.800403e+10,17503006845144,8.6,51.6,12.9,04/01/2022


In [52]:
product_info.columns = [x.replace(" ", "") if x != "BottleVolume(ml)" else "BottleVolume" for x in product_info.columns]

In [55]:
product_info.drop(['UPC', 'SCC', 'ReportDate'], inplace = True, axis = 1)

In [57]:
product_info.to_csv('./data/liquor_product_clean.csv')